# Просьба пока не смотреть. Дорабатываю.

Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/
Реализовать самостоятельно логистическую регрессию
Обучить ее методом градиентного спуска
Методом nesterov momentum
Методом rmsprop
В качестве dataset'а взять Iris, оставив 2 класса:
Iris Versicolor
Iris Virginica

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
from math import exp, ceil, floor
from scipy.spatial import distance
from matplotlib import pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
iris = datasets.load_iris()

In [3]:
X = pd.DataFrame(iris.data)
y = pd.DataFrame(iris.target)

In [4]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [5]:
df = pd.DataFrame(iris.data)

In [6]:
df['y'] = pd.DataFrame(y)

In [7]:
df = df[(df['y'] == 1) | (df['y'] == 2)]

In [8]:
y = pd.DataFrame(df.y)
X = pd.DataFrame(df.drop('y', axis=1))

In [9]:
X.shape

(100, 4)

In [10]:
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X))

In [11]:
X.shape

(100, 4)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=7)

In [13]:
X_train.shape

(80, 4)

In [14]:
y_train.shape

(80, 1)

In [15]:
data = pd.DataFrame(np.hstack([y_train, X_train]))

In [16]:
data.shape

(80, 5)

## Вероятностный взгляд. Бинарная классификация
Рассмотрим задачу классификации. Предположим, что ответ нашей модели $h_\theta(x_i)$ получен на основе логистической регрессии ($\sigma(Wx_i + b)$). Ее значения лежат в диапазоне от 0 до 1, что может быть интерпретировано, как вероятность, что $x_i$ принадлежит positive классу. Если вероятность $< 0.5$ - мы делаем предсказание о принадлежности отрицательному классу, а если $>= 0.5$ - положительному. Следовательно:

$$p(y_i = 1 \vert x_i)  = h_\theta(x_i)$$
$$p(y_i = 0 \vert x_i) = 1 - h_\theta(x_i)$$

Мы можем соединить это в одно уравнение:

$$p(y_i | x_i) = [h_\theta(x_i)]^{(y_i)} [1 - h_\theta(x_i)]^{(1 - y_i)}$$

Опять-таки из предположения, что наши данные независимы и одинаково распределены перейдем к правдоподобию:

$$L(x, y) = \prod_{i = 1}^{N}[h_\theta(x_i)]^{(y_i)} [1 - h_\theta(x_i)]^{(1 - y_i)}$$

Точно так же, как в случае MSE, возмьмем логарифм и инвертируем знак. Получим наш loss:

$$J = -\sum_{i=1}^{N} y_i\log (h_\theta(x_i)) + (1 - y_i)\log(1 - h_\theta(x_i))$$

In [38]:
# Реализуем алгоритм подбора весов логистической регрессии методом градиентного спуска
def GradientDescentLogReg(C,data):
    print('C=%d' %C)
    errorAccuracy=10**-5
    weights=[0., 0., 0., 0.]
    k=0.1#длина шага
    l=data[0].count()#количество элементов в выборке
    distance_euclidean=0

    weightsDelta=[0., 0., 0., 0.]
    for step in range(10000):
        oldweightsDelta=weightsDelta
        weightsDelta=[0., 0., 0., 0.]

        for obj in data.values:
            y=obj[0]
            gradient=y*(1-1/(1+exp(-y*(weights[0]*obj[1]+weights[1]*obj[2]+weights[2]*obj[3]+weights[3]*obj[4]))))
            weightsDelta=list(map(lambda w,wd,x: wd+x*gradient-k*C*w,weights,weightsDelta,obj[1:]))

        #проверим Евклидово расстояние между соседними шагами, если менее порога, выходим из цикла
        distance_euclidean=distance.euclidean(weightsDelta,oldweightsDelta)
        if distance_euclidean<errorAccuracy:
            print('Дошли до заданной ошибки точности на шаге: %d' % step)
            break

        weights=list(map(lambda w,wd: w+wd*k/l,weights,weightsDelta))

    print('Евклидово расстояние=%.6f' %distance_euclidean)
    return weights

In [39]:
W = GradientDescentLogReg(1, data)

C=1
Дошли до заданной ошибки точности на шаге: 506
Евклидово расстояние=0.000010


In [40]:
#Проверим адекватность метода на стандартном алгоритме "из коробки"

In [41]:
logreg = LogisticRegression()

In [42]:
logreg.fit(X, y)

/home/pavel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
logreg.coef_

array([[-0.28109636, -0.59297763,  2.21267607,  2.38820582]])

In [44]:
logreg.intercept_

array([0.08787821])

In [45]:
y_pred = logreg.predict(X_test)

In [46]:
y_pred

array([1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1])

In [47]:
accuracy_score(y_test, y_pred)

0.95

In [48]:
#Реализуем функцию прогнозирования принадлежности к классу
def func(W, X):
    klass = []
    for i in X.values:
        sg = (1/(1+exp(W[0]*i[0]+W[1]*i[1]+W[2]*i[2]+W[3]*i[3])))
        if sg < 0.5:
            a = 2
        else: 
            a = 1
        klass.append(a)
    return klass

In [49]:
y_pred = func(W, X_test)

In [50]:
y_pred

[1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1]

In [51]:
accuracy_score(y_test, y_pred)

0.95

In [ ]:
def linear_interpolation(X, idx):
    idx_min = floor(idx)
    idx_max = ceil(idx)
    if idx_min == idx_max or idx_max >= len(X):
        return X[idx_min]
    elif idx_min < 0:
        return X[idx_max]
    else:
        return X[idx_min] + (idx - idx_min)*X[idx_max]

def EDM(X, gamma, lr=0.25):
    Y = []
    v = 0
    for x in X:
        v = gamma*v + lr*x
        Y.append(v)
    return np.asarray(Y)

def NM(X, gamma, lr=0.25):
    Y = []
    v = 0
    for i in range(len(X)):
        v = gamma*v + lr*(linear_interpolation(X, i+gamma*v) if i+gamma*v < len(X) else 0)
        Y.append(v)
    return np.asarray(Y)

def SmoothedNM(X, gamma, lr=0.25):
    Y = []
    v = 0
    for i in range(len(X)):
        lookahead4 = linear_interpolation(X, i+gamma*v/4)   if i+gamma*v/4      < len(X) else 0
        lookahead3 = linear_interpolation(X, i+gamma*v/2)   if i+gamma*v/2      < len(X) else 0
        lookahead2 = linear_interpolation(X, i+gamma*v*3/4) if i+gamma*v*3/4    < len(X) else 0
        lookahead1 = linear_interpolation(X, i+gamma*v)     if i+gamma*v        < len(X) else 0
        v = gamma*v + lr*(lookahead4 + lookahead3 + lookahead2 + lookahead1)/4
        Y.append(v)
    return np.asarray(Y)

def Adagrad(X, eps, lr=2.5):
    Y = []
    G = 0
    for x in X:
        G += x*x
        v = lr/np.sqrt(G + eps)*x
        Y.append(v)
    return np.asarray(Y)

def RMSProp(X, gamma, lr=0.25, eps=0.00001):
    Y = []
    EG = 0
    for x in X:
        EG = gamma*EG + (1-gamma)*x*x
        v = lr/np.sqrt(EG + eps)*x
        Y.append(v)
    return np.asarray(Y)

def Adadelta(X, gamma, lr=50.0, eps=0.001):
    Y = []
    EG = 0
    EDTheta = lr
    for x in X:
        EG = gamma*EG + (1-gamma)*x*x
        v = np.sqrt(EDTheta + eps)/np.sqrt(EG + eps)*x
        Y.append(v)
        EDTheta = gamma*EDTheta + (1-gamma)*v*v
    return np.asarray(Y)

def AdadeltaZeroStart(X, gamma, eps=0.001):
    return Adadelta(X, gamma, lr=0.0, eps=eps)

def AdadeltaBigStart(X, gamma, eps=0.001):
    return Adadelta(X, gamma, lr=50.0, eps=eps)

def Adam(X, beta1, beta2=0.999, lr=0.25, eps=0.0000001):
    Y = []
    m = 0
    v = 0
    for i, x in enumerate(X):
        m = beta1*m + (1-beta1)*x
        v = beta2*v + (1-beta2)*x*x
        m_hat = m/(1- pow(beta1, i+1) )
        v_hat = v/(1- pow(beta2, i+1) )
        dthetha = lr/np.sqrt(v_hat + eps)*m_hat
        Y.append(dthetha)
    return np.asarray(Y)

np.random.seed(413)
X = np.arange(0, 300)

D_Thetha_spikes = np.asarray( [int(x%60 == 0) for x in X])
D_Thetha_rectangles = np.asarray( [2*int(x%40 < 20) - 1 for x in X])
D_Thetha_noisy_sin = np.asarray( [np.sin(x/20) + np.random.random() - 0.5 for x in X])
D_Thetha_very_noisy_sin = np.asarray( [np.sin(x/20)/5 + np.random.random() - 0.5 for x in X])
D_Thetha_uneven_sawtooth = np.asarray( [ x%20/(15*int(x > 80) + 5) for x in X])
D_Thetha_saturation = np.asarray( [ int(x % 80 < 40) for x in X])

for method_label, method, parameter_step in [
                ("GRAD_Simple_Momentum", EDM, [0.25, 0.9, 0.975]),
                ("GRAD_Nesterov", NM, [0.25, 0.9, 0.975]),
                ("GRAD_Smoothed_Nesterov", SmoothedNM, [0.25, 0.9, 0.975]),
                ("GRAD_Adagrad", Adagrad, [0.0000001, 0.1, 10.0]),
                ("GRAD_RMSProp", RMSProp, [0.25, 0.9, 0.975]),
                ("GRAD_AdadeltaZeroStart", AdadeltaZeroStart, [0.25, 0.9, 0.975]),
                ("GRAD_AdadeltaBigStart", AdadeltaBigStart, [0.25, 0.9, 0.975]),
                ("GRAD_Adam", Adam, [0.25, 0.9, 0.975]),
            ]:
    for label, D_Thetha in [("spikes", D_Thetha_spikes),
                            ("rectangles", D_Thetha_rectangles),
                            ("noisy sin", D_Thetha_noisy_sin),
                            ("very noisy sin", D_Thetha_very_noisy_sin),
                            ("uneven sawtooth", D_Thetha_uneven_sawtooth),
                            ("saturation", D_Thetha_saturation), ]:
        fig = plt.figure(figsize=[16.0, 9.0])
        ax = fig.add_subplot(111)

        ax.plot(X, D_Thetha, label="gradient")
        for gamma in parameter_step:
            Y = method(D_Thetha, gamma)
            ax.plot(X, Y, label="param="+str(gamma))

        ax.spines['bottom'].set_position('zero')
        full_name = method_label + "_" + label

        plt.xticks(np.arange(0, 300, 20))
        plt.grid(True)
        plt.title(full_name)
        plt.xlabel('epoch')
        plt.ylabel('value')
        plt.legend()
        plt.show(block=True) #Uncoomment and comment next line if you just want to watch
        plt.savefig(full_name)
        plt.close(fig)